<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=500 height=300 align=center class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Spring 2024<br>
<font color=3C99D size=5>
    Practical Assignment 3 - Heart Disease Prediction using Hidden Markov Models  <br>
<font color=696880 size=4>
Omid Daliran


# Personal Data

In [1]:
# Set your student number and name
student_number = '401106182'
Name = 'Amirhossein'
Last_Name = 'Souri'

# Libraries

In [2]:
import csv
import numpy as np
import pandas as pd
import copy

# Q2: Heart Disease Prediction using Hidden Markov  (100 Points)

# Introduction


In this notebook, we explore the application of Hidden Markov Models (HMM) in predicting heart disease risk based on DNA sequences. Heart disease is a prevalent and life-threatening condition, and early detection is crucial for effective management. By leveraging HMM, we aim to identify regions within DNA sequences associated with high and low GC content, which have been linked to heart disease susceptibility.


# Hidden Markov Models (HMM) (40 points)

Hidden Markov Models are probabilistic models widely used in sequential data analysis, particularly in bioinformatics. They are characterized by states, transitions between states, and emission probabilities associated with each state. In our context, the states represent the underlying biological characteristics of DNA sequences, while the emission probabilities signify the likelihood of observing specific nucleotides given each state.

To facilitate our heart disease classification, we'll implement a custom Hidden Markov Model (HMM) class in Python. This class will include the Viterbi and Forward algorithms.

In [3]:
class HMM:
    def __init__(self, states: list, emissions: dict, start_probabilities: list, transition_probabilities: list):
        self.states = states
        self.state_labels = {i: state for i, state in enumerate(self.states)}
        self.emissions = emissions
        self.start_probabilities = start_probabilities
        self.transition_probabilities = transition_probabilities

    def viterbi(self, sequence: str) -> tuple:
        path, max_probability = [], 0
        
        path, probs = self.recursive_viterbi(sequence)
        path = ''.join(path)
        max_probability = max(probs)

        return path, max_probability
    
    def recursive_viterbi(self, sequence: str) -> tuple:
        path, probs = [], []
        n = len(sequence) - 1
        
        if n == 0:
            for i in range(len(self.states)):
                probs.append(self.emissions[sequence[0]][i] * self.start_probabilities[i])
            path.append(self.state_labels[np.argmax(np.array(probs))])
            return path, probs
        
        path, previous_probs = self.recursive_viterbi(sequence[:-1])
        for i in range(len(self.states)):
            prob = self.emissions[sequence[n]][i]
            max_value = -1
            for j in range(len(self.states)):
                value = previous_probs[j] * self.transition_probabilities[j][i]
                if value > max_value:
                    max_value = value
            prob *= max_value
            probs.append(prob)
            
        path.append(self.state_labels[np.argmax(np.array(probs))])
        
        return path, probs

    
    def forward_algorithm(self, sequence: str) -> dict:
        probs = []
        
        n = len(sequence) - 1
        
        if n == 0:
            for i in range(len(self.states)):
                probs.append(self.emissions[sequence[0]][i] * self.start_probabilities[i])
            probability_last_states = {self.states[i]: probs[i] for i in range(len(self.states))}
            return probability_last_states
        
        previous_probs = list(self.forward_algorithm(sequence[:-1]).values())
        for i in range(len(self.states)):
            prob = self.emissions[sequence[n]][i]
            sum_value = 0
            for j in range(len(self.states)):
                sum_value += previous_probs[j] * self.transition_probabilities[j][i]
            prob *= sum_value
            probs.append(prob)
            
        denumerator = sum(probs)
        for i in range(len(self.states)):
            probs[i] /= denumerator
            
        probability_last_states = {self.states[i]: probs[i] for i in range(len(self.states))}

        return probability_last_states

GC content, the proportion of guanine (G) and cytosine (C) nucleotides in a DNA sequence, is a fundamental genomic feature. Variations in GC content have been implicated in various biological processes, including gene regulation and disease susceptibility. Specifically, high GC content (H) regions are associated with increased gene expression and potential regulatory elements, while low GC content (L) regions may indicate structural elements or regions prone to mutations. The transition and emission and start probabilities are as shown in the image below:

![Probabilities](Probabilities.png)

In [4]:
# TODO: Complete and run this code to check your implementation

states = ['H', 'L']
emissions = {'A': [0.2, 0.3], 'C': [0.3, 0.2], 'G': [0.3, 0.2], 'T': [0.2, 0.3]}
start_probabilities = [0.5, 0.5]
transition_probabilities = [[0.5, 0.5], [0.4, 0.6]]

hmm = HMM(states, emissions, start_probabilities, transition_probabilities)

# Example DNA sequence
sequence = "ATGCGCGATCGATCGAATCGCGT"

# Viterbi algorithm
path, max_probability = hmm.viterbi(sequence)
print("Viterbi Path:", path)
print("Max Probability (Viterbi):", max_probability)

# Forward algorithm
probability = hmm.forward_algorithm(sequence)
print("Probability (Forward Algorithm):", probability)

Viterbi Path: LLHHHHHLLHHLLHHLLLHHHHL
Max Probability (Viterbi): 1.1438396227480495e-19
Probability (Forward Algorithm): {'H': 0.35818008853498684, 'L': 0.6418199114650132}


# Calculating Probabilities for Blood Type, Height, and Weight (10 points)

After obtaining the sequence of H and L regions from the DNA using the Viterbi algorithm, we proceed to calculate the probabilities for other attributes such as blood type, height, weight, and the presence or absence of DCM (DCM, or Dilated Cardiomyopathy, is a heart condition characterized by the enlargement of the heart's left ventricle. It can lead to heart failure if not properly managed.) using the Forward algorithm. In this step, we aim to leverage the Hidden Markov Model (HMM) to infer these attributes based on the observed GC content sequence.


We will use the functionality of the Forward algorithm to compute the probabilities associated with each possible state after observing the sequence.

Attributes and Possible States:
* Blood Type: O, A, B, AB
* Height: 160 cm, 170 cm, 180 cm, 190 cm
* Weight: 50 kg, 65 kg, 80 kg, 95 kg
* DCM: Y (indicating presence) or N (indicating absence)


### Attention:
I have changed the format of emission dictionaries, in order to match them with my HMM Class implementation.

**No data has been changed**

In [5]:
def probabilities_from_sequence(sequence: str) -> tuple:

    # Define the emission probabilities for each feature

#     height_emissions = [{'H': 0.2, 'L': 0.8}, {'H': 0.3, 'L': 0.7},
#                           {'H': 0.9, 'L': 0.1}, {'H': 0.8, 'L': 0.2}]

#     weight_emissions = [{'H': 0.3, 'L': 0.7}, {'H': 0.1, 'L': 0.9},
#                           {'H': 0.2, 'L': 0.8}, {'H': 0.7, 'L': 0.3}]

#     blood_type_emissions = [{'H': 0.75, 'L': 0.25}, {'H': 0.8, 'L': 0.2},
#                               {'H': 0.85, 'L': 0.15}, {'H': 0.7, 'L': 0.3}]
    
#     DCM_emissions = [{'H': 0.7, 'L': 0.3}, {'H': 0.6, 'L': 0.4}]
    
    height_emissions = {'H': [0.2, 0.3, 0.9, 0.8], 'L': [0.8, 0.7, 0.1, 0.2]}
    weight_emissions = {'H': [0.3, 0.1, 0.2, 0.7], 'L': [0.7, 0.9, 0.8, 0.3]}
    blood_type_emissions = {'H': [0.75, 0.8, 0.85, 0.7], 'L': [0.25, 0.2, 0.15, 0.3]}
    DCM_emissions = {'H': [0.7, 0.6], 'L': [0.3, 0.4]}


    # Define the transition probabilities for each feature
    height_transition_probabilities = [[0.7, 0.2, 0.05, 0.05],
                                       [0.1, 0.8, 0.05, 0.05],
                                       [0.1, 0.1, 0.7, 0.1],
                                       [0.1, 0.1, 0.1, 0.7]]
    weight_transition_probabilities = [[0.7, 0.2, 0.05, 0.05],
                                       [0.1, 0.8, 0.05, 0.05],
                                       [0.1, 0.1, 0.7, 0.1],
                                       [0.1, 0.1, 0.1, 0.7]]
    blood_type_transition_probabilities = [[0.7, 0.2, 0.05, 0.05],
                                           [0.1, 0.8, 0.05, 0.05],
                                           [0.1, 0.1, 0.7, 0.1],
                                           [0.1, 0.1, 0.1, 0.7]]
    DCM_transition_probabilities = [[0.9, 0.1], [0.1, 0.9]]

    # Define start probabilities for each state
    height_start_probabilities = [0.1, 0.4, 0.4, 0.1]
    weight_start_probabilities = [0.15, 0.35, 0.35, 0.15]
    blood_type_start_probabilities = [0.1, 0.35, 0.35, 0.2]
    DCM_start_probabilities = [0.25, 0.75]

    height_probability_dic, weight_probability_dic, blood_type_probability_dic, DCM_probability_dic = None, None, None, None

    height_hmm = HMM([160, 170, 180, 190], height_emissions,
                     height_start_probabilities, height_transition_probabilities)
    height_probability_dic = height_hmm.forward_algorithm(sequence)
    
    weight_hmm = HMM([50, 65, 80, 95], weight_emissions,
                     weight_start_probabilities, weight_transition_probabilities)
    weight_probability_dic = weight_hmm.forward_algorithm(sequence)
    
    blood_type_hmm = HMM(['O', 'A', 'B', 'AB'], blood_type_emissions,
                     blood_type_start_probabilities, blood_type_transition_probabilities)
    blood_type_probability_dic = blood_type_hmm.forward_algorithm(sequence)
    
    DCM_hmm = HMM(['Y', 'N'], DCM_emissions,
                     DCM_start_probabilities, DCM_transition_probabilities)
    DCM_probability_dic = DCM_hmm.forward_algorithm(sequence)

    return height_probability_dic, weight_probability_dic, blood_type_probability_dic, DCM_probability_dic

In [6]:
# Run this code to check your implementation
# Example sequence of 'H' and 'L'
sequence = "HHLLHHHLLHLLLLHHLL"

height_probability, weight_probability, blood_type_probability, DCM_probability = probabilities_from_sequence(sequence)

print("Probability of height given sequence:", height_probability)
print("Probability of weight given sequence:", weight_probability)
print("Probability of blood type given sequence:", blood_type_probability)
print("Probability of DCM given sequence:", DCM_probability)

Probability of height given sequence: {160: 0.3660593174871113, 170: 0.5760951509787053, 180: 0.015850353367627773, 190: 0.04199517816655549}
Probability of weight given sequence: {50: 0.2845952908892867, 65: 0.42707075390249144, 80: 0.20957356322963908, 95: 0.07876039197858277}
Probability of blood type given sequence: {'O': 0.29628284733090365, 'A': 0.3305080573190477, 'B': 0.08789883086305705, 'AB': 0.28531026448699165}
Probability of DCM given sequence: {'Y': 0.32977581810083306, 'N': 0.670224181899167}


# Bayesian Network for Heart Disease Probability Calculation (30 points)

In this section, we introduce a Bayesian network to calculate the probability of heart disease based on various features. We assume that the relationships between features follow a specific structure, depicted in the image below.

![Bayesian Net](BN.png)

The Bayesian network comprises nodes representing different features, including sex, weight, height, Dilated Cardiomyopathy (DCM), blood type, and heart disease. The directed edges between nodes indicate probabilistic dependencies between them.

Function Implementation:
We will implement two functions to facilitate the calculation of heart disease probability:

1. calculate_sex_probability:

  This function calculates the probability of sex based on the given probabilities of weight and height.

2. calculate_heart_disease_probability:

  This function computes the probability of heart disease based on the probabilities of sex, DCM, and blood type.

In [7]:
# The probability of sex given height and weight, the tuple is in this order: (sex, height, weight)
P_sex_given_hw = {
    ('M', 160, 50): 0.7, ('F', 160, 50): 0.3,
    ('M', 160, 65): 0.2, ('F', 160, 65): 0.8,
    ('M', 160, 80): 0.3, ('F', 160, 80): 0.7,
    ('M', 160, 95): 0.2, ('F', 160, 95): 0.8,
    ('M', 170, 50): 0.8, ('F', 170, 50): 0.2,
    ('M', 170, 65): 0.75, ('F', 170, 65): 0.25,
    ('M', 170, 80): 0.4, ('F', 170, 80): 0.6,
    ('M', 170, 95): 0.3, ('F', 170, 95): 0.7,
    ('M', 180, 50): 0.9, ('F', 180, 50): 0.1,
    ('M', 180, 65): 0.7, ('F', 180, 65): 0.3,
    ('M', 180, 80): 0.65, ('F', 180, 80): 0.35,
    ('M', 180, 95): 0.4, ('F', 180, 95): 0.6,
    ('M', 190, 50): 0.95, ('F', 190, 50): 0.05,
    ('M', 190, 65): 0.8, ('F', 190, 65): 0.2,
    ('M', 190, 80): 0.6, ('F', 190, 80): 0.4,
    ('M', 190, 95): 0.95, ('F', 190, 95): 0.05
}

In [8]:
def calculate_sex_probability(height_prob: dict, weight_prob: dict) -> dict:
    
    sex_given_hw = copy.deepcopy(P_sex_given_hw)
    final = {}
    
#     join
    for x in sex_given_hw.keys():
        sex_given_hw[x] *= height_prob[x[1]]
        
#     sumout
    for x in [('M', 50), ('F', 50), ('M', 65), ('F', 65), ('M', 80), ('F', 80), ('M', 95), ('F', 95)]:
        sum_value = 0
        for y in sex_given_hw.keys():
            if y[0] == x[0] and y[2] == x[1]:
                sum_value += sex_given_hw[y]
        final[x] = sum_value
    sex_given_hw = final
    final = {}
    
#     join
    for x in sex_given_hw.keys():
        sex_given_hw[x] *= weight_prob[x[1]]
    
#     sumout
    for x in ['M', 'F']:
        sum_value = 0
        for y in sex_given_hw.keys():
            if y[0] == x[0]:
                sum_value += sex_given_hw[y]
        final[x] = sum_value
    sex_given_hw = final
    

    return sex_given_hw

In [9]:
# Run this code to test you implementation
# Example probabilities of height
P_height = {160: 0.1, 170: 0.3, 180: 0.4, 190: 0.2}
# Example probabilities of weight
P_weight = {50: 0.2, 65: 0.3, 80: 0.4, 95: 0.1}
# Example conditional probabilities of sex given height and weight


sex_probabilities = calculate_sex_probability(P_height, P_weight)
print("Probability of sex:", sex_probabilities)

Probability of sex: {'M': 0.6355000000000002, 'F': 0.36450000000000005}


In [10]:
# The probability of hear disease given sex, DCM and blood type, the tuple is in this order: (heart disease, sex, blood type, DCM)
P_heart_disease_given_parents = {
    ('N', 'M', 'O', 'Y'): 0.3, ('Y', 'M', 'O', 'Y'): 0.7,
    ('N', 'M', 'A', 'Y'): 0.4, ('Y', 'M', 'A', 'Y'): 0.6,
    ('N', 'M', 'B', 'Y'): 0.5, ('Y', 'M', 'B', 'Y'): 0.5,
    ('N', 'M', 'AB', 'Y'): 0.6, ('Y', 'M', 'AB', 'Y'): 0.4,
    ('N', 'F', 'O', 'Y'): 0.6, ('Y', 'F', 'O', 'Y'): 0.4,
    ('N', 'F', 'A', 'Y'): 0.3, ('Y', 'F', 'A', 'Y'): 0.7,
    ('N', 'F', 'B', 'Y'): 0.4, ('Y', 'F', 'B', 'Y'): 0.6,
    ('N', 'F', 'AB', 'Y'): 0.7, ('Y', 'F', 'AB', 'Y'): 0.3,
    ('N', 'M', 'O', 'N'): 0.6, ('Y', 'M', 'O', 'N'): 0.4,
    ('N', 'M', 'A', 'N'): 0.15, ('Y', 'M', 'A', 'N'): 0.85,
    ('N', 'M', 'B', 'N'): 0.4, ('Y', 'M', 'B', 'N'): 0.6,
    ('N', 'M', 'AB', 'N'): 0.7, ('Y', 'M', 'AB', 'N'): 0.3,
    ('N', 'F', 'O', 'N'): 0.8, ('Y', 'F', 'O', 'N'): 0.2,
    ('N', 'F', 'A', 'N'): 0.7, ('Y', 'F', 'A', 'N'): 0.3,
    ('N', 'F', 'B', 'N'): 0.65, ('Y', 'F', 'B', 'N'): 0.35,
    ('N', 'F', 'AB', 'N'): 0.35, ('Y', 'F', 'AB', 'N'): 0.65,
}

In [11]:
def calculate_heart_disease_probability(DCM_prob: dict, sex_prob: dict, blood_type_prob: dict) -> dict:
    
    hd_given_parents = copy.deepcopy(P_heart_disease_given_parents)
    final = {}
    
#     join
    for x in hd_given_parents.keys():
        hd_given_parents[x] *= sex_prob[x[1]]
        
#     sumout
    for x in [('N', 'O', 'Y'), ('Y', 'O', 'Y'), ('N', 'A', 'Y'), ('Y', 'A', 'Y'), 
              ('N', 'B', 'Y'), ('Y', 'B', 'Y'), ('N', 'AB', 'Y'), ('Y', 'AB', 'Y'),
              ('N', 'O', 'N'), ('Y', 'O', 'N'), ('N', 'A', 'N'), ('Y', 'A', 'N'), 
              ('N', 'B', 'N'), ('Y', 'B', 'N'), ('N', 'AB', 'N'), ('Y', 'AB', 'N')]:
        sum_value = 0
        for y in hd_given_parents.keys():
            if y[0] == x[0] and y[2] == x[1] and y[3] == x[2]:
                sum_value += hd_given_parents[y]
        final[x] = sum_value
    hd_given_parents = final
    final = {}
    
#     join
    for x in hd_given_parents.keys():
        hd_given_parents[x] *= blood_type_prob[x[1]]
        
#     sumout
    for x in [('N', 'Y'), ('Y', 'Y'), ('N', 'N'), ('Y', 'N')]:
        sum_value = 0
        for y in hd_given_parents.keys():
            if y[0] == x[0] and y[2] == x[1]:
                sum_value += hd_given_parents[y]
        final[x] = sum_value
    hd_given_parents = final
    final = {}
    
#     join
    for x in hd_given_parents.keys():
        hd_given_parents[x] *= DCM_prob[x[1]]
        
#     sumout
    for x in ['Y', 'N']:
        sum_value = 0
        for y in hd_given_parents.keys():
            if y[0] == x[0]:
                sum_value += hd_given_parents[y]
        final[x] = sum_value
    hd_given_parents = final
    final = {}

    
    return hd_given_parents

In [12]:
# Run this code to test you implementation
# Example probabilities of DCM
P_DCM = {'Y': 0.2, 'N': 0.8}
# Example probabilities of blood type
P_blood_type = {'O': 0.4, 'A': 0.3, 'B': 0.2, 'AB': 0.1}
# Example probabilities of sex
P_sex = {'M': 0.3, 'F': 0.7}


heart_disease_probabilities = calculate_heart_disease_probability(P_DCM, P_sex, P_blood_type)
print("Probability of heart disease: ",heart_disease_probabilities)

Probability of heart disease:  {'Y': 0.4152, 'N': 0.5848}


# Loading DNA Sequences and Heart Disease Probability Calculation (20 points)

In this part, we will load the DNA sequences from the file DNA_patients.csv and calculate the probability of heart disease for each patient. Subsequently, we will classify the patients based on their heart disease probability and save the data.

In [13]:
# Read the CSV file into a DataFrame
df = pd.read_csv('DNA_patients.csv')

df.head()

,ID,DNA
0,1,CTCCAATACCCCCCACAAGAACACACCCATAAAATTGCAACCCACA...
1,2,TTGATGTAGAAGTATATTTGTTGGGTATTTGAGGTAACGTTATTAG...
2,3,ATATAATTTAAAGTCACTGGAAAAAAACAACCTAATAAAAACCACC...
3,4,TAAAGACAAAATTAAATTGAAGTAATGTTATGTTAAAATTTTGAAT...
4,5,AGCGTTTGTTCGTTAGCCGTAGGCAATGACGTGATTCAGGTCTGTG...


In [14]:
# Extract DNA sequences and store them in a list
DNAs = df['DNA']

states = ['H', 'L']
emissions = {'A': [0.2, 0.3], 'C': [0.3, 0.2], 'G': [0.3, 0.2], 'T': [0.2, 0.3]}
start_probabilities = [0.5, 0.5]
transition_probabilities = [[0.5, 0.5], [0.4, 0.6]]

hmm = HMM(states, emissions, start_probabilities, transition_probabilities)

GC_content_seqs = []

for seq in DNAs:
    path, max_probability = hmm.viterbi(seq)
    GC_content_seqs.append(path)

In [15]:
heart_disease_classified = []
for seq in GC_content_seqs:
    h, w, bt, dcm = probabilities_from_sequence(seq)
    s = calculate_sex_probability(h, w)
    hd = calculate_heart_disease_probability(dcm, s, bt)
    if hd['Y'] >= 0.5:
        heart_disease_classified.append(1)
    else:
        heart_disease_classified.append(0)

In [16]:
# Write the result to CSV
labels = [(i+1, heart_disease_classified[i]) for i in range(len(heart_disease_classified))]
csv_filename = "my_heart_disease_result.csv"
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'Label'])
    for id, label in labels:
        writer.writerow([id, label])

**Note: You should upload heart_disease_result.csv along side your notebook.**